In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pickle as pkl
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import zscore
import copy

import datetime as dt
from datetime import datetime
import warnings
warnings.filterwarnings(action = 'ignore')
import seaborn as sns
import tqdm
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.cluster import KMeans
import rrcf

---
#### Anomaly Score 계산

In [ ]:
save = {}

for i in range(len(vision_chunk)): 

    results = pd.DataFrame(vision_chunk[i][column].reset_index(drop=True))
    results['DATE'] = vision_chunk[i]['DATE_ms'].reset_index(drop=True)
    
    
    n = len(vision_chunk[i][column])

    num_trees = 100
    tree_size = 256

    # Construct forest
    forest = []
    while len(forest) < num_trees:
        # Select random subsets of points uniformly from point set
        ixs = np.random.choice(n, size=(n // tree_size, tree_size),
                            replace=False)
        # Add sampled trees to forest
        trees = [rrcf.RCTree(results[column].values.reshape(-1,1)[ix], index_labels=ix) for ix in ixs]
        forest.extend(trees)

    # Compute average CoDisp
    avg_codisp = pd.Series(0.0, index=np.arange(n))
    index = np.zeros(n)
    for tree in forest:
        codisp = pd.Series({leaf : tree.codisp(leaf) for leaf in tree.leaves})
        avg_codisp[codisp.index] += codisp
        np.add.at(index, codisp.index.values, 1)
    avg_codisp /= index
    
    
    # diff = 200
    cnt = 25
    # cnt = len(results)//diff
    diff = len(results)//cnt
    print(cnt)
    results['anomaly'] = avg_codisp
    test = results[['anomaly']].groupby(results.index//diff).mean()
 
    save[vision_chunk[i]['chunk_no'].unique()[0]] = test